<a href="https://colab.research.google.com/github/harikuts/federated-learning-trials/blob/master/DecentralizedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This notebook contains the reproduction of results of the original paper on federated learning.

## Plan

The roadmap for development is as follows:
*   Construct standard MNIST example.
*   To be continued.




In [ ]:
import tensorflow as tf
print(tf.__version__)
# !pip install tensorflow==2.1-rc0

2.1.0-rc0


# Standard MNIST

There are baseline implementations of a standard example of MNIST. A Keras implementation staands as the first example, but we will port this over to Tensorflow as it provides more low-level functionality.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

# tf.enable_eager_execution()

# Import MNIST data
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# Predictions
predictions = model(x_train[:1]).numpy()
# Softmax
tf.nn.softmax(predictions).numpy()

# Defining the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

# Compile model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'], validation_data=(x_test, y_test))

# Fit model
model.fit(x_train, y_train, epochs=16)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



TypeError: ignored

In [ ]:
# print(model.get_weights()[0].shape)
# print(model.get_weights()[1].shape)
# print(model.get_weights()[2].shape)
# print(model.get_weights()[3].shape)

import numpy as np
a = np.array([1, 2, 3, 4])

b = a + a
b = sum([a,a])
print(b)
b = b / 2
print(b)

[2 4 6 8]
[1. 2. 3. 4.]


# Experimental Approaches

## Customization Functions

In this section, you can develop and select the dataset and models you want to use.

Please note that your selected dataset and selected model must be comaptible (check input and output layers on the model).

**The modules in this section must be run before running the experiments as they contain dataset and model building functions.**

### Dataset Grab Functions

Develop dataset functions here, including any pre-processing that needs to be done. Then set `get_dataset()` to utilize your function of  choice.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import pdb

# Use this function to select one of the dataset grab functions
def get_dataset():
  return get_mnist()

# MNIST Dataset
def get_mnist():
  # Import MNIST data
  print ("\nDownloading MNIST data...")
  mnist = tf.keras.datasets.mnist
  # Load data into trains
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  return x_train, x_test, y_train, y_test

### Model Creation Functions
Develop models here. Note that you may have to create specific input/output layers here to match your dataset.

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import pdb

# Use this function to select one of the model creation functions
def create_model():
  return standardNN()

# Standard Neural Network
def standardNN():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
  return model

## Federated Learning Validation

### Network Model
Here we use nodes to carry models. The reason for doing this to prevent the instantiation of new models each time weights have to be transferred. Instead, the state of each model can be preserved in the node that it resides in.

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import pdb

# Used to start execution ASAP
# tf.enable_eager_execution()

# Configuration
num_clients = 8
num_epochs = 2
num_server_rounds = 8
num_client_rounds = 4
nonIID = False
predeterminedSplit = True
dataSplit = []
print ("Configuration:" + \
       "\n\t%d clients." % (num_clients) + \
       "\n\t%d training epochs." % (num_epochs)  + \
       "\n\tUsing %sIID data." % ("non-" if nonIID else ""))

# Server class
class Server:
  def __init__(self, modelGenerator):
    self.model = modelGenerator()
    self.clients = []
    self.neighbors = []
# Client class
class Client:
  def __init__(self, modelGenerator):
    self.model = modelGenerator()
    self.neighbors = []
    self.x_data = None
    self.y_data = None
    self.data_size = None
  def plotAccuracy(self, histories):
    # Compile histories
    categorical_accuracy = []
    val_categorical_accuracy = []
    for history in histories:
      categorical_accuracy = categorical_accuracy + history.history['acc']
      # val_categorical_accuracy = val_categorical_accuracy + history.history['val_categorical_accuracy']
    # The history of our accuracy during training.
    plt.plot(categorical_accuracy)
    plt.plot(val_categorical_accuracy)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Number of epochs')
    plt.legend(['train', 'validation'], loc='upper left')
    return plt
  def train(self):
    history = self.model.fit(self.x_data, self.y_data, epochs=num_epochs)
    # print(history.history.keys())
    # self.accPlot = self.plotAccuracy([history])

# Weight averaging
def averageWeights(weightsList, weighting=None):
  denominator = len(weightsList)
  new_weights = []
  if weighting is None:
    # Handle IID data (balanced)
    for part in range(len(weightsList[0])):
      part_stack = [weights[part] for weights in weightsList]
      new_stack = sum(part_stack) / denominator
      new_stack = np.array(new_stack)
      new_weights.append(new_stack)
    return new_weights
  else:
    for part in range(len(weightsList[0])):
      part_stack = [weights[part] for weights in weightsList]
      # part_stack = np.array(part_stack) * weighting
      for i in range(len(weighting)):
        part_stack[i] = part_stack[i] * weighting[i]
      new_stack = sum(part_stack)
      new_stack = np.array(new_stack)
      new_weights.append(new_stack)
    return new_weights


# Create the network
print ("\nCreating a network...")
server = Server(create_model)
for i in range(num_clients):
  server.clients.append(Client(create_model))

# Load data
x_train, x_test, y_train, y_test = get_dataset()

# Splitting the dataset for different clients
print ("\nSplitting data into different clients...")
if nonIID:
  print ("\tRandomly assigning ranges of data...")
  percentageMarkers = []
  for i in range(num_clients-1):
    percentageMarkers.append(random.random())
  percentageMarkers.append(1.0)
  percentageMarkers = sorted(percentageMarkers)
else:
  print ("\tUniformly assigning ranges of data")
  percentageMarkers = [1/num_clients * (n+1) for n in range(num_clients)]
# Storing each subset of data in a client
print ("\tStoring subsets of data into each client...")
xMarkers = [int(marker * len(x_train)) for marker in percentageMarkers]
yMarkers = [int(marker * len(y_train)) for marker in percentageMarkers]
for j in range(len(percentageMarkers)):
  server.clients[j].x_data = x_train[(xMarkers[j-1] if j > 0 else 0):xMarkers[j]]
  server.clients[j].y_data = y_train[(yMarkers[j-1] if j > 0 else 0):yMarkers[j]]
  server.clients[j].data_size = len(server.clients[j].x_data)

# Client data diagnostic
print ("\nFinished setting up client data!")
for client in server.clients:
  print ("\tClient %d:\tX: %d\tY: %d" % (server.clients.index(client), len(client.x_data), len(client.y_data)))

# Server action
server_accuracies = []
server_losses = []
for server_round in range(num_server_rounds):
  print("\nSERVER ROUND ", server_round, ":\n")
  # Save server model weights
  global_weights = server.model.get_weights()
  # Clients' actions
  client_weight_list = []
  for client in server.clients:
    print("\nCLIENT ", server.clients.index(client), ":\n")
    # Initialize recorded weights
    round_weight_list = []
    for client_round in range(num_client_rounds):
      # Accept global weights
      client.model.set_weights(global_weights)
      # Train
      client.train()
      # Record weights
      round_weight_list.append(client.model.get_weights())
    client_weight_list.append(averageWeights(round_weight_list))
  client_data_sizes = [client.data_size for client in server.clients]
  client_weighting = np.array(client_data_sizes) / sum(client_data_sizes)
  server.model.set_weights(averageWeights(client_weight_list, weighting=client_weighting))
  loss, acc = server.model.evaluate(x_test, y_test)
  print("\nSERVER ROUND ", server_round, " ACCURACY: ", acc, "\n")
  server_accuracies.append(acc)
  server_losses.append(loss)
  print("FINAL RESULTS:\nAccuracies: ", server_accuracies, "\nLoss: ", server_losses)

235/235 [==============================] - 0s 2ms/step - loss: 0.5225 - accuracy: 0.8503
Epoch 1/2
235/235 [==============================] - 0s 2ms/step - loss: 1.0547 - accuracy: 0.6804
Epoch 2/2
235/235 [==============================] - 0s 2ms/step - loss: 0.5189 - accuracy: 0.8499

CLIENT  6 :

Epoch 1/2
235/235 [==============================] - 0s 2ms/step - loss: 1.0545 - accuracy: 0.6772
Epoch 2/2
235/235 [==============================] - 0s 2ms/step - loss: 0.5309 - accuracy: 0.8428
Epoch 1/2
235/235 [==============================] - 0s 2ms/step - loss: 1.0585 - accuracy: 0.6756
Epoch 2/2
235/235 [==============================] - 0s 2ms/step - loss: 0.5263 - accuracy: 0.8424
Epoch 1/2
235/235 [==============================] - 0s 2ms/step - loss: 1.0297 - accuracy: 0.6868
Epoch 2/2
235/235 [==============================] - 0s 2ms/step - loss: 0.5215 - accuracy: 0.8436
Epoch 1/2
235/235 [==============================] - 0s 2ms/step - loss: 1.0305 - accuracy: 0.6812
Epoch 

KeyboardInterrupt: ignored

## Decentralized Learning

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import pdb

# Used to start execution ASAP
# tf.enable_eager_execution()

# Configuration
num_clients = 8
num_epochs = 2
num_learning_rounds = 16
num_client_rounds = 1
link_reliability = 0.75
nonIID = False
print ("Configuration:" + \
       "\n\t%d clients." % (num_clients) + \
       "\n\t%d training epochs." % (num_epochs)  + \
       "\n\tUsing %sIID data." % ("non-" if nonIID else ""))

# Client class
class Client:
  def __init__(self, modelGenerator):
    self.model = modelGenerator()
    self.neighbors = []
    self.x_data = None
    self.y_data = None
    self.data_size = None
    self.accuracy_history = []
    self.loss_history = []
  def plotAccuracy(self, histories):
    # Compile histories
    categorical_accuracy = []
    val_categorical_accuracy = []
    for history in histories:
      categorical_accuracy = categorical_accuracy + history.history['acc']
    # The history of our accuracy during training.
    plt.plot(categorical_accuracy)
    plt.plot(val_categorical_accuracy)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Number of epochs')
    plt.legend(['train', 'validation'], loc='upper left')
    return plt
  def train(self):
    history = self.model.fit(self.x_data, self.y_data, epochs=num_epochs)
    self.setOutgoingMessage()
    # print(history.history.keys())
    # self.accPlot = self.plotAccuracy([history])
  def test(self, x, y):
    loss, acc = self.model.evaluate(x, y, verbose=1)
    self.accuracy_history.append(acc)
    self.loss_history.append(loss)
    return loss, acc
  def setOutgoingMessage(self):
    self.outgoing_message = (self.data_size, self.model.get_weights())
  def communityLearn(self):
    # Accept incoming messages
    incoming_messages = []
    for neighbor in self.neighbors:
      # Link reliability functionality
      r = random.random()
      if (r <= link_reliability) or (clientList.index(neighbor) == clientList.index(self)):
        incoming_messages.append(neighbor.outgoing_message)
    # Grab sizes, set ratios, grab weights
    ordered_sizes = [message[0] for message in incoming_messages]
    ordered_sizes = np.array(ordered_sizes) / sum(ordered_sizes)
    ordered_weights = [message[1] for message in incoming_messages]
    # Compute new weights
    new_weights = averageWeights(ordered_weights, ordered_sizes)
    # Create new model with appropriate weights
    self.model = create_model()
    self.model.set_weights(new_weights)

# Weight averaging
def averageWeights(weightsList, weighting=None):
  denominator = len(weightsList)
  new_weights = []
  if weighting is None:
    # Handle IID data (balanced)
    for part in range(len(weightsList[0])):
      part_stack = [weights[part] for weights in weightsList]
      new_stack = sum(part_stack) / denominator
      new_stack = np.array(new_stack)
      new_weights.append(new_stack)
    return new_weights
  else:
    for part in range(len(weightsList[0])):
      part_stack = [weights[part] for weights in weightsList]
      # part_stack = np.array(part_stack) * weighting
      for i in range(len(weighting)):
        part_stack[i] = part_stack[i] * weighting[i]
      new_stack = sum(part_stack)
      new_stack = np.array(new_stack)
      new_weights.append(new_stack)
    return new_weights


# # Create a strongly connected network
# print ("\nCreating a network...")
# clientList = []
# for i in range(num_clients):
#   clientList.append(Client(create_model))
# # Add neighbors
# for client in clientList:
#   for neighbor in clientList:
#     client.neighbors.append(neighbor)

# Create a weakly connected network
print ("\nCreating a weakly connected network...")
clientList = []
for i in range(8):
  clientList.append(Client(create_model))
# Add linkings
clientList[0].neighbors += [clientList[0], clientList[2], clientList[3], clientList[4]]
clientList[1].neighbors += [clientList[1], clientList[2]]
clientList[2].neighbors += [clientList[0], clientList[1], clientList[2], clientList[3]]
clientList[3].neighbors += [clientList[0], clientList[2], clientList[3], clientList[4], clientList[7]]
clientList[4].neighbors += [clientList[0], clientList[3], clientList[4], clientList[5], clientList[6], clientList[7]]
clientList[5].neighbors += [clientList[4], clientList[5], clientList[6]]
clientList[6].neighbors += [clientList[4], clientList[5], clientList[6], clientList[7]]
clientList[7].neighbors += [clientList[3], clientList[4], clientList[6], clientList[7]]

# Load data
x_train, x_test, y_train, y_test = get_dataset()

# Splitting the dataset for different clients
print ("\nSplitting data into different clients...")
if nonIID:
  print ("\tRandomly assigning ranges of data...")
  percentageMarkers = []
  for i in range(num_clients-1):
    percentageMarkers.append(random.random())
  percentageMarkers.append(1.0)
  percentageMarkers = sorted(percentageMarkers)
else:
  print ("\tUniformly assigning ranges of data")
  percentageMarkers = [1/num_clients * (n+1) for n in range(num_clients)]
# Storing each subset of data in a client
print ("\tStoring subsets of data into each client...")
xMarkers = [int(marker * len(x_train)) for marker in percentageMarkers]
yMarkers = [int(marker * len(y_train)) for marker in percentageMarkers]
for j in range(len(percentageMarkers)):
  clientList[j].x_data = x_train[(xMarkers[j-1] if j > 0 else 0):xMarkers[j]]
  clientList[j].y_data = y_train[(yMarkers[j-1] if j > 0 else 0):yMarkers[j]]
  clientList[j].data_size = len(clientList[j].x_data)

# Client data diagnostic
print ("\nFinished setting up client data!")
for client in clientList:
  print ("\tClient %d:\tX: %d\tY: %d" % (clientList.index(client), len(client.x_data), len(client.y_data)))

for learning_round in range(num_learning_rounds):
  print("\nLEARNING ROUND ", learning_round, ":\n")
  # Have each client learn on its data
  for client in clientList:
    print ("\nROUND", learning_round, ", CLIENT", clientList.index(client), "TRAINING\n")
    client.train()
  # Communicate and learn
  for client in clientList:
    print ("\nROUND", learning_round, ", CLIENT", clientList.index(client), "LEARNING\n")
    client.communityLearn()
  # Test at the end of this round
  for client in clientList:
    print ("\nROUND", learning_round, ", CLIENT", clientList.index(client), "TESTING\n")
    client.test(x_test, y_test)
# Print out results
for client in clientList:
  print("Client ", clientList.index(client), ":\n")
  print("\tAccuracy History: ", client.accuracy_history, "\n")
  print("\tLoss History: ", client.loss_history, "\n")